<a href="https://colab.research.google.com/github/StankoDiego/SOA_EA_2/blob/main/HPC/Stanko_Diego_ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Introducción

# 2. Armado del ambiente

## 2.1 Armado del ambiente en CPU

No se requiere ejecuciones previas para la ejecución del programa

##2.2 Armado del ambiente en GPU

Se requiere la instalación del modulo de Cuda para Python

In [2]:
!pip install pycuda


     |████████████████████████████████| 1.6MB 10.1MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621008 sha256=219603135469f17b91a78ec35f1dedbc7ce6ddbad35225ad3bbcf2d40182d7b7
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=f3359b7b2123a23d2c2e4e578742306354b3d068d738bbf903c61e9c77cfa126
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3. Desarrollo

##3.1 Desarrollo en CPU

In [32]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio =  10#@param {type: "number"}
orden_taylor = 5#@param {type: "number"}
punto_evalacion =  3#@param {type: "number"}
# --------------------------------------------

import numpy
import time
import math

tiempoTotal = time.time() * 1000

coeficientes = numpy.random.randint(10, size = orden_polinomio +1)

longitud_poli = len(coeficientes)

derivadas = numpy.arange(longitud_poli)

print("Coeficientes:\n", coeficientes)

polin_taylor = numpy.zeros(orden_taylor, dtype=float)
longitud_taylor = len(polin_taylor)

tiempoBucle = time.time() *1000
#----------------------ALGORITMO----------------------------
for idx_t in range(0, longitud_taylor):
  valor = 0
  aux = 0
  idx_p = idx_t

  if idx_t == 0:
    aux += coeficientes[idx_p]
    idx_p = idx_t + 1
    for idx in range(idx_t + 1, longitud_poli):
      aux += coeficientes[idx] * math.pow(punto_evalacion, derivadas[idx])
    valor = aux
    polin_taylor[idx_t] = valor 
  else:
    if idx_t < longitud_poli:
      aux += coeficientes[idx_p]
      for idx in range(idx_t + 1, longitud_poli):
        termino = coeficientes[idx] * derivadas[idx]
        coeficientes[idx] = termino
        derivadas[idx] = derivadas[idx] - 1
        aux += termino * math.pow(punto_evalacion, derivadas[idx])
    valor = aux
    polin_taylor[idx_t] = valor 
#----------------------FINALIZACION----------------------------

tiempoBucle = (time.time() *1000) - tiempoBucle
print("Termino el algoritmo")
print(polin_taylor)

tiempoTotal = (time.time() *1000) - tiempoTotal
print("Tiempo del bucle: " , tiempoBucle , "[ms.]")
print("Tiempo del total: " , tiempoTotal , "[ms.]")

Coeficientes:
 [3 0 7 5 2 2 6 9 3 6 6]
Termino el algoritmo
[  516981.  1643640.  4718030. 12071064. 27086520.]
Tiempo del bucle:  0.36279296875 [ms.]
Tiempo del total:  1.407958984375 [ms.]


##3.2 Desarrollo en GPU

In [28]:
# --------------------------------------------
#@title 3.1 Parámetros de ejecución { vertical-output: true }

orden_polinomio =  20#@param {type: "number"}
orden_taylor = 1#@param {type: "number"}
punto_evalacion =  3#@param {type: "number"}
# --------------------------------------------

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

import time
import numpy

tiempoTotal = (time.time() * 1000)

# Vectores en CPU.
#coeficientes_cpu = numpy.random.randint(10, size = orden_polinomio + 1 )
coeficientes_cpu = numpy.array([3,2,3,4])
coeficientes_cpu = coeficientes_cpu.astype(numpy.int32())

print("Coeficientes: ")
print(coeficientes_cpu)

terminos_taylor_cpu = numpy.zeros(orden_taylor + 1, dtype=int)
terminos_taylor_cpu = terminos_taylor_cpu.astype(numpy.int32())

resultado_cpu = numpy.empty_like(terminos_taylor_cpu)
#resultado_cpu = resultado_cpu.astype(numpy.int32())

longitud = len(coeficientes_cpu)
derivadas_cpu = numpy.arange(longitud)
derivadas_cpu = derivadas_cpu.astype(numpy.int32())

print("Derivadas")
print(derivadas_cpu)
# Vectores en GPU.
coeficientes_gpu = cuda.mem_alloc(coeficientes_cpu.nbytes)
terminos_taylor_gpu = cuda.mem_alloc(terminos_taylor_cpu.nbytes)
derivadas_gpu = cuda.mem_alloc(derivadas_cpu.nbytes)

# Copio los datos
cuda.memcpy_htod(coeficientes_gpu, coeficientes_cpu)
cuda.memcpy_htod(terminos_taylor_gpu, terminos_taylor_cpu)
cuda.memcpy_htod(derivadas_gpu, derivadas_cpu)

# CPU - Defino la función kernel que ejecutará en GPU.
module = SourceModule("""
__global__ void kernel_taylor( int punto, 
                               int long_pol, 
                               int *C, 
                               int *T,
                               int *D){
  
  int idx = threadIdx.x + blockIdx.x*blockDim.x;
  
  int valor = 0;
  int idx_polinomio = 0;
  int aux = 0;  
  
  if(idx == 0){
    valor += C[idx_polinomio];
    for(idx_polinomio = idx + 1; idx_polinomio < long_pol + 1; idx_polinomio++){
      aux += C[idx_polinomio] * pow((double) punto, (double) D[idx_polinomio]); 
    }
    valor += aux;
  } else 
  {
    valor += C[idx_polinomio];

    for(idx_polinomio = idx + 1; idx_polinomio < long_pol + 1; idx_polinomio++){
      int termino = C[idx_polinomio] * D[idx_polinomio];
      C[idx_polinomio] = termino;
      D[idx_polinomio] = D[idx_polinomio] - 1;
      aux += termino * pow((double) punto, (double) D[idx_polinomio]);
    } 
    valor += aux;  
  }
  T[idx] = valor; 
}
""")
# CPU - Genero la función kernel.
kernel = module.get_function("kernel_taylor")

tiempo_gpu = (time.time() * 1000)

# GPU - Ejecuta el kernel.
# TODO: Falta consultar limites del GPU, para armar las dimensiones correctamente.
dim_hilo = 256
dim_bloque = numpy.int( (orden_taylor + 1 + dim_hilo-1) / dim_hilo )

#TODO: Ojo, con los tipos de las variables en el kernel.
kernel(numpy.int32(punto_evalacion),
       numpy.int32(longitud),
       coeficientes_gpu, terminos_taylor_gpu, derivadas_gpu, 
       block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1))

cuda.memcpy_dtoh(resultado_cpu, terminos_taylor_gpu)

tiempo_gpu = (time.time() * 1000) - tiempo_gpu

tiempo_total = (time.time() * 1000) - tiempoTotal
print(resultado_cpu)
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )
print("Tiempo CPU: ", tiempo_total, "[ms]" )
print("Tiempo GPU: ", tiempo_gpu, "[ms]" )

Coeficientes: 
[3 2 3 4]
Derivadas
[0 1 2 3]
[597 246]
Thread x:  256 , Bloque x: 1
Thread x:  256 , Bloque x: 1
Tiempo CPU:  3.44677734375 [ms]
Tiempo GPU:  0.283203125 [ms]


#4. Tabla de pasos

#5. Conclusiones